# Boost.Python's Custom Converters

Boost.Python is a C++ library which enables seamless interoperability between C++ and the Python programming language. It allows you to quickly and seamlessly expose C++ classes, functions, and objects to Python, and vice-versa, using no special tools -- just your C++ compiler.

One of the powerful features of Boost.Python is the ability to write custom converters. These converters allow you to convert Python objects to C++ objects and vice versa. This is particularly useful when you want to expose complex C++ data structures to Python or when you want to use Python data structures in your C++ code.

In this notebook, we will explore the concept of custom converters in Boost.Python and illustrate their usage with numerous examples.

## Basic Concept of Custom Converters

In Boost.Python, a converter is a piece of code that knows how to convert a C++ object to a Python object and vice versa. The library provides built-in converters for many standard C++ and Python types. However, for user-defined types or more complex conversions, you may need to write a custom converter.

A custom converter in Boost.Python consists of two parts:

1. **Python to C++ converter**: This part of the converter takes a Python object and converts it into a C++ object. This is done by implementing a `convertible` function that checks if the Python object can be converted to the C++ type, and a `construct` function that performs the actual conversion.

2. **C++ to Python converter**: This part of the converter takes a C++ object and converts it into a Python object. This is done by implementing a `to_python_value` function that performs the conversion.

Once the converter is implemented, it can be registered with Boost.Python using the `boost::python::to_python_converter` and `boost::python::converter::registry::push_back` functions.

## Example: Converting a Python List to a C++ Vector

Let's start with a simple example. We will create a custom converter that converts a Python list to a C++ `std::vector<int>`.

First, we need to include the necessary headers and use the required namespaces:

```cpp
#include <boost/python.hpp>
#include <vector>

using namespace boost::python;
```

Next, we define the Python to C++ converter. We implement the `convertible` function to check if the Python object is a list, and the `construct` function to convert the list to a `std::vector<int>`:

```cpp
struct ListToVector
{
    static void* convertible(PyObject* obj_ptr)
    {
        if (!PyList_Check(obj_ptr)) return 0;
        return obj_ptr;
    }

    static void construct(PyObject* obj_ptr, converter::rvalue_from_python_stage1_data* data)
    {
        list l(handle<>(borrowed(obj_ptr)));
        void* storage = ((converter::rvalue_from_python_storage<std::vector<int>>*)data)->storage.bytes;
        new (storage) std::vector<int>();
        std::vector<int>& v = *((std::vector<int>*)storage);
        int len = len(l);
        for (int i = 0; i < len; ++i)
        {
            v.push_back(extract<int>(l[i]));
        }
        data->convertible = storage;
    }
};
```

Finally, we register the converter with Boost.Python:

```cpp
BOOST_PYTHON_MODULE(example)
{
    converter::registry::push_back(
        &ListToVector::convertible,
        &ListToVector::construct,
        type_id<std::vector<int>>());
}
```

Now, we can pass Python lists to C++ functions that take `std::vector<int>` arguments, and Boost.Python will automatically use our custom converter to convert the lists to vectors.

## Example: Converting a C++ Vector to a Python List

Now let's create a custom converter that converts a C++ `std::vector<int>` to a Python list. This will allow us to return C++ vectors from functions and have them automatically converted to Python lists.

First, we need to include the necessary headers and use the required namespaces:

```cpp
#include <boost/python.hpp>
#include <vector>

using namespace boost::python;
```

Next, we define the C++ to Python converter. We implement the `to_python_value` function to convert the `std::vector<int>` to a Python list:

```cpp
struct VectorToList
{
    static PyObject* convert(const std::vector<int>& v)
    {
        list l;
        for (int i = 0; i < v.size(); ++i)
        {
            l.append(v[i]);
        }
        return incref(l.ptr());
    }
};
```

Finally, we register the converter with Boost.Python:

```cpp
BOOST_PYTHON_MODULE(example)
{
    to_python_converter<std::vector<int>, VectorToList>();
}
```

Now, we can return `std::vector<int>` objects from C++ functions, and Boost.Python will automatically use our custom converter to convert the vectors to Python lists.

## Example: Converting a Python Dictionary to a C++ Map

For a more complex example, let's create a custom converter that converts a Python dictionary to a C++ `std::map<std::string, int>`.

First, we need to include the necessary headers and use the required namespaces:

```cpp
#include <boost/python.hpp>
#include <map>

using namespace boost::python;
```

Next, we define the Python to C++ converter. We implement the `convertible` function to check if the Python object is a dictionary, and the `construct` function to convert the dictionary to a `std::map<std::string, int>`:

```cpp
struct DictToMap
{
    static void* convertible(PyObject* obj_ptr)
    {
        if (!PyDict_Check(obj_ptr)) return 0;
        return obj_ptr;
    }

    static void construct(PyObject* obj_ptr, converter::rvalue_from_python_stage1_data* data)
    {
        dict d(handle<>(borrowed(obj_ptr)));
        void* storage = ((converter::rvalue_from_python_storage<std::map<std::string, int>>*)data)->storage.bytes;
        new (storage) std::map<std::string, int>();
        std::map<std::string, int>& m = *((std::map<std::string, int>*)storage);
        list keys = d.keys();
        for (int i = 0; i < len(keys); ++i)
        {
            std::string key = extract<std::string>(keys[i]);
            int value = extract<int>(d[key]);
            m[key] = value;
        }
        data->convertible = storage;
    }
};
```

Finally, we register the converter with Boost.Python:

```cpp
BOOST_PYTHON_MODULE(example)
{
    converter::registry::push_back(
        &DictToMap::convertible,
        &DictToMap::construct,
        type_id<std::map<std::string, int>>());
}
```

Now, we can pass Python dictionaries to C++ functions that take `std::map<std::string, int>` arguments, and Boost.Python will automatically use our custom converter to convert the dictionaries to maps.

## Conclusion

Custom converters in Boost.Python are a powerful tool that allows you to seamlessly convert between Python and C++ data structures. By implementing and registering custom converters, you can pass complex data structures between Python and C++ and take full advantage of the features of both languages.

In this notebook, we have seen how to create custom converters for converting Python lists to C++ vectors, C++ vectors to Python lists, and Python dictionaries to C++ maps. The same principles can be applied to create custom converters for any other data structures you may need to use in your Python and C++ code.